In [1]:

import numpy as np

import matplotlib.pyplot as plt
from scipy import integrate

from plot_configs import set_plot_params
cmaps = set_plot_params()
%config InlineBackend.figure_format = 'retina'


# <center>Mean velocity profile in stably stratified channel turbulence</center>

             Created on: July 26, 2025
           Developed by: Rene Pecnik (r.pecnik@tudelft.nl)
                         Pedro Costa (psimoescosta1@tudelft.nl)
                         Sanath Kotturshettar (s.b.kotturshettar@tudelft.nl)
                         Process & Energy Department, Faculty of 3mE
                         Delft University of Technology, the Netherlands.

This notebook supports the reference ***"Mean velocity profile in stably stratified channel turbulence"*** with the numerical implementation of,
 *(1)* the composite velocity profile derived in this reference based on Monin–Obukhov Similarity Theory (MOST), and 
 *(2)* the corresponding numerical integration of this profile to compute the skin friction coefficient. 
 These can be computed for varying combinations of friction Reynolds and Richardson numbers ($Re_{\tau}$ and $Ri_{\tau}$).
#


# <center> Monin-Obukhov Similarity Theory (MOST)  </center>

The most established framework used to predict the mean flow in stratified turbulent flows in the context of atmospheric boundary layers is the Monin-Obukhov Similarity Theory. In MOST, mean velocity gradients $({\mathrm{d}\overline{u}}/{\mathrm{d}z})$ in the overlap region of a fully developed turbulent boundary layer subjected to stratification are related to the wall-normal distance $(z)$ and the Obukhov length scale $(L)$ as, 

$$\frac{\kappa z}{u_{\tau}} \frac{\mathrm{d}\overline{u}}{\mathrm{d}z} = \phi\left(\frac{z}{L}\right);$$
#
#
with Obukhov length scale ($L$) defined as,

$$ L = -\frac{u_{\tau}^{3/2}}{\kappa\ ({g}/{\theta_0})\ q_{\mathrm{w}}}. $$
#
$L$ estimates the height above which buoyancy starts to quench turbulent mixing, because buoyancy destruction of turbulence overwhelms shear production.
#
#
### Local Obukhov length scale ($\Lambda$)

Since the scope of $L$ was limited to the 'surface layer' in atmospheric boundary layers, [Nieuwstadt (1984)](https://journals.ametsoc.org/view/journals/atsc/41/14/1520-0469_1984_041_2202_ttsots_2_0_co_2.xml) reasoned that the dynamics of stratified turbulence are better characterized by local rather than surface quantities beyond the surface layer, and introduced a scale based on local momentum and heat fluxes. For pressure-driven turbulent channel flows, with walls at fixed temperatures, the local length scale can be expressed in terms of $L$ as,



$$ \Lambda(z) = L \left(1 - \frac{z}{h}\right)^{3/2} $$

# <center> Estimating the mean velocity profile </center>


The mean velocity profile in stably stratified turbulent channel flows is estimated as a function of the friction Reynolds ($Re_{\tau}$) and the friction Richardson number ($Ri_{\tau}$). The composite profile is comprised of 5 distinct regions:

### <u>I.  Viscous sublayer: </u>

$$u^+ = z^+.$$

$$ \ $$
### <u> II. Inertial sublayer: </u>

 $$u^+ = \frac{1}{\kappa} \ln(z^+) + \frac{4.7}{\kappa} \frac{2h}{L} \left(1 - \frac{z}{h}\right)^{-1/2} + C_1,$$

where,

$$C_1 = {z_\mathrm{M}^+} - \frac{1}{\kappa}\ln({z_\mathrm{M}^+}) - \frac{4.7}{\kappa} \frac{2h}{L} - \frac{4.7}{\kappa} \frac{h}{L}\frac{{z_\mathrm{M}^+}}{Re_{\tau}},\ $$
where $z_{\mathrm{M}}^+$ is the point of intersection of turbulent and viscous stresses close to the wall, which is estimated as;
$${z_\mathrm{M}^+} = 11.1 + 2 \ \mathrm{max}\left(0, \frac{Ri_{\tau}}{Re_{\tau}} - 0.5\right).\ $$


The inertial sublayer extends from $z^+ \approx 30 $ to the the transition point, $z_t$. The transition point can be analytically estimated as the location where $z \approx \Lambda$. Therefore, 

$$z_t = \left.z\right|_{z/\Lambda = 1}. $$

The stratified outer layer starts from this point.
$$ \ $$
 
### <u> III. Stratified outer layer: </u>


$$u^+ = \frac{1}{\kappa}\ln{\frac{z}{h}}-\frac{4.7}{\kappa} \frac{h}{L} \ln{\left(1 - \frac{z}{h}\right)} + \frac{2}{\kappa} \sqrt{1 - \frac{z}{h}} - \frac{2}{\kappa} \ln{\left(1 + \sqrt{1 - \frac{z}{h}}\right)} + C_2$$

where,

$$ C_2 \approx \frac{1}{\kappa} \ln{Re_{\tau}} + 5.2 $$

The stratified outer layer extends upto the point where viscous stresses become comparable to turbulent stresses. This is observed to occur at $z \approx 8\Lambda$. Therefore we have,

$$ z_{III}^u = \left.z\right|_{z/\Lambda = 8} $$

$$ \ $$
### <u> IV. Turbulent-viscous transition layer: </u>

Within the turbulent-viscous transition layer, both viscous and turbulent diffusion play comparable roles. Due to the limited extent of this region, blending the adjacent layers provides a reasonable estimate of the flow velocity here.

$$ \ $$
### <u> V. Viscous core: </u>

$$u_{\mathrm{cl}}^+ - u^+ = \frac{Re_{\tau}}{2}\left(1 - \frac{z}{h}\right)^2,$$

where the centerline velocity is estimated through parametrizing the deviation from the neutral case. If the centerline velocity at given $Re_{\tau}$, is $u_{\mathrm{cl,N}}^+ \approx {1}/{\kappa} \ln Re_{\tau} + 5.2$, then stratification suppresses turbulence near the centerline that leads to an increased $u_{\mathrm{cl}}^+$ that is captured by:

$$\Delta u_{\mathrm{cl}}^+ = u_{\mathrm{cl}}^+ - u_{\mathrm{cl,N}}^+ = Ri_{\tau}^{0.63} Re_{\tau}^{-0.12}.$$

The mean velocity follows parabolic profile until wherever the turbulent stresses pick up from being zero at the centerline and become comparable to viscous stresses. This point is estimated to scale as,

$$ \frac{z^l_{\mathrm{V}}}{h} =  1 - \frac{1}{\sqrt{Re_L}}\ \sqrt{\frac{4.7}{2\kappa}}. $$


$$ \ $$

### Estimating $h/L$

For estimating the Obukhov length scale, $L$, we use the relation,

$$\frac{h}{L} = 0.115 \left(\frac{Ri_{\tau}}{\sqrt{Re_{\tau}}}\right)^{2/3}.$$



#
#
#
#
The following code snippet defines the ```MOST_Ch``` class, which contains methods for reconstructing velocity profiles in different flow regions based on the equations outlined above.
#
#
#
#

In [2]:
class MOST_Ch:

    def __init__(self, ReTau, RiTau):
        
        self.ReTau = ReTau
        self.RiTau = RiTau

        self.L     = (.115*(self.RiTau/self.ReTau**(1./2.))**(2./3.))**(-1)
        self.u_cl  = 1/.41*np.log(self.ReTau) + 5.2 + 1.*(self.RiTau)**(0.63)*self.ReTau**(-0.12)

        coeff  = [1, (1./self.L)**2 - 3, 3, -1]
        coeff2 = [1, (1./(8*self.L))**2 - 3, 3, -1]
        roots  = np.roots(coeff)
        roots2 = np.roots(coeff2)

        self.int_c0 = 11.1 + 2.*max(0., self.RiTau/self.ReTau - 0.5)

        for jj in range(len(roots)):

            if np.isreal(roots[jj]):
                self.zt  = np.real(roots[jj])
            if np.isreal(roots2[jj]):
                self.z3u = np.real(roots2[jj])
    
        self.z5l  = 1-np.sqrt(4.7*0.5/.41*1./self.ReTau/self.L)

    def uI(self, z):

        zp = z*self.ReTau

        return zp

    def uII(self, z):

        zp = z*self.ReTau

        c0_c0  = self.int_c0 - 1/.41*np.log(self.int_c0) - 9.4*1./0.41*1/self.L - 4.7*1./0.41*1/self.L*self.int_c0/self.ReTau

        return 1/0.41*(np.log(zp) + 9.4/self.L*(1- zp/self.ReTau)**(-1./2.)) + c0_c0
    
    def uIII(self, z):

        return 1/0.41*np.log(z) \
                - 4.7/0.41*1./self.L*(np.log(1-z)) \
                + 2./0.41*(1 -z)**(.5) \
                - 2./0.41*np.log(1 + (1 -z)**(.5)) \
                + 1/0.41*np.log(self.ReTau) \
                + 5.2
    
    def uV(self, z):

        return -1./2.*self.ReTau*(1 - z)**2 + self.u_cl
    
    def uIV(self, z):

        u3u = self.uIII(self.z3u)
        u5l = self.uV  (self.z5l)

        return u3u + (u5l - u3u) * (z - self.z3u) / (self.z5l - self.z3u)
    
    def composite_u(self, z = None, N = 500):

        if z is None:
            z  = np.linspace(0, 1., N)
                    
        u = np.zeros_like(z)

        mask1 = z <= self.int_c0/self.ReTau
        mask2 = (z > self.int_c0/self.ReTau) & (z <= self.zt)
        mask3 = (z > self.zt) & (z <= self.z3u)
        mask4 = (z > self.z3u) & (z <= self.z5l)
        mask5 = z > self.z5l

        u[mask1] = self.uI(z[mask1])
        u[mask2] = self.uII(z[mask2])
        u[mask3] = self.uIII(z[mask3])
        u[mask4] = self.uIV(z[mask4])
        u[mask5] = self.uV(z[mask5])

        return z, u

The following snippet provides an interactive mean velocity profile with sliders for $Re_{\tau}$ and $Ri_{\tau}$. Adjusting the sliders updates the profile corresponding to the selected parameter values and also prints the estimate of the skin friction coefficient.

In [3]:
from ipywidgets import interactive, FloatSlider
from IPython.display import display

def plot_profile(reTau, riTau):

    profile = MOST_Ch(reTau, riTau)
    
    z, u = profile.composite_u()

    plt.figure()
    plt.plot(z,u, lw = 1.5, color = 'k')
    plt.plot(profile.zt, profile.uII(profile.zt), ls ='', marker = '^', ms = 19, markeredgecolor = 'crimson', color = 'crimson', alpha= 0.3)
    plt.plot(profile.z3u, profile.uIII(profile.z3u), ls ='', marker = 'v', ms = 19, markeredgecolor = 'orangered', color = 'orangered', alpha= 0.3)
    plt.plot(profile.z5l, profile.uV(profile.z5l), ls ='', marker = 'P', ms = 19, markeredgecolor = 'palevioletred', color = 'palevioletred', alpha= 0.3)

    plt.xlim([0, 1])
    plt.xlabel(r'$z/h$')
    plt.ylabel(r'$u^+$')
    plt.ylim([1e-6, max(u) + 5])

    print("Estimated skin friction coefficient = ", 2./(integrate.trapezoid(u, z))**2 )

plt_prof = interactive(
    plot_profile,
    reTau=FloatSlider(min=0, max=2000, step=10, value=550, description='Re\u03C4'),
    riTau=FloatSlider(min=1e-3, max=2000, step=10, value=720, description='Ri\u03C4')
)

display(plt_prof)

plt_prof.kwargs

interactive(children=(FloatSlider(value=550.0, description='Reτ', max=2000.0, step=10.0), FloatSlider(value=72…

{'reTau': 550.0, 'riTau': 720.0}